# HOW TO ADD NEW SPECIES TO `EXOSPHID`

## Main Concept

The Exospheric Photolysis model is configured to support only a limited number of atomic and molecular species by default. These include:

- Water ($H_2O$)  
- Hydroxyl ($OH$)  
- Molecular Hydrogen ($H_2$)  
- Atomic Hydrogen ($H$)  
- Ionized (negative) Hydrogen ($H^{-}$)  
- Hydroperoxyl ($HO_2$)  
- Hydrogen Peroxide ($H_2O_2$)  
- Helium ($He$)  
- Neon ($Ne$)  

So far, the model has been applied primarily for investigations involving the Moon. This explains the focus on hydrogen- and oxygen-bearing species (for studying the lunar water cycle) and noble gases such as Neon and Helium, which are the main constituents of the Moon's exosphere.  

Technically, any species of interest could be added to the exospheric model for any exospheric investigations or for studying potential exospheres of other celestial bodies. In this notebook, two methods are suggested for adding new species.

## Example: Helium

In [83]:
if !isdefined(Main, :EXOSPHID)
    include(joinpath(@__DIR__, "..", "src", "EXOSPHID.jl"))
    using .EXOSPHID
end

**Methodology**

- Literature sources will usually provide database for wavelength dependent absorption cross-section data, which will be different for every species.
- We already have solar flux values at 1 A.U. given by W.F. Huebner (1992) in photons $cm^{-2}$ $s^{-1}$.
- First, we shall obtain the species dependent fluxes expressed in number of photons per unit time for the new species we want to add to `EXOSPHID`
- To do that, we will need to reference the data to the reference wavelength bins (`lambda_phi`) provided by W.F. Huebner (1992)
- Then, we shall do the numerical product of both quantities
- Additionnally, we may calculate the photodestruction rate 

#### Solar flux expressed in $cm^{-2} s^{-1}$ per wavelength bin from Huebner (2015)

In [84]:
# Wavelength values
lambda_phi = [
    1, 2, 4, 6, 8, 10, 40, 50, 60, 70, 80, 90, 100, 110, 120, 128, 153, 176, 205, 231, 270, 280, 300, 320, 340, 360, 370, 400, 430, 460, 480, 500, 
    520, 540, 560, 580, 600, 620, 630, 680, 710, 740, 770, 800, 830, 860, 890, 911, 920, 930, 940, 950, 960, 970, 980, 995, 1007, 1017, 1027, 1035, 1045, 1057, 1070, 1080, 
    1090, 1100, 1110, 1120, 1130, 1140, 1150, 1157, 1163, 1170, 1176, 1183, 1190, 1198, 1205, 1212, 1220, 1227, 1235, 1242, 1250, 1258, 1266, 1274, 1282, 1290, 1299, 1307, 1316, 1324, 1333, 1342,
    1351, 1360, 1370, 1379, 1389, 1408, 1428, 1449, 1470, 1492, 1515, 1538, 1562, 1587, 1613, 1639, 1667, 1695, 1724, 1739, 1754, 1770, 1786, 1802, 1818, 1835, 1852, 1869, 1887, 1905, 1923, 1942, 
    1961, 1980, 2000, 2020, 2041, 2062, 2083, 2105, 2128, 2150, 2174, 2198, 2222, 2247, 2273, 2299, 2326, 2353, 2381, 2410, 2439, 2469, 2500, 2532, 2564, 2597, 2632, 2667, 2703, 2740, 2778, 2817,
    2857, 2899, 2941, 2985, 3030, 3077, 3125, 3175, 3225, 3275, 3325, 3375, 3425, 3475, 3525, 3575, 3625, 3675, 3725, 3775, 3825, 3875, 3925, 3975, 4025, 4075, 4125, 4175, 4225, 4275, 4325, 4375, 4425, 
    4475, 4525, 4575, 4625, 4675, 4725, 4775, 4825, 4875, 4925, 4975, 5025, 5075, 5125, 5175, 5225, 5275, 5325, 5375, 5425, 5475, 5525, 5575, 5625, 5675, 5725, 5775, 5825, 5875, 5925, 5975, 6025, 6075, 
    6125, 6175, 6225, 6275, 6325, 6375, 6425, 6475, 6550, 6650, 6750, 6850, 6950, 7050, 7150, 7250, 7350, 7450, 7550, 7650, 7750, 7850, 7950, 8050, 8150, 8250, 8350, 8450, 8550, 8650, 8750, 8850, 8950, 
    9050, 9150, 9250, 9350, 9450, 9550, 9650, 9750, 9850, 9950, 10150, 10750, 11250, 11750, 12250, 12750, 13250, 13750, 14250, 14750, 15250, 15750, 16250, 16750, 17250, 17750, 18250, 18750, 19250, 19750, 20375, 21500, 22500, 
    23500, 24500, 25500, 26500, 27500, 28500, 29500, 30500, 31500, 32500, 33500, 34500, 35500, 36500, 37500, 38500, 39500, 40500, 41500, 42500, 43500, 44500, 45500, 46500, 47500, 48500, 49500, 52750, 65000, 75000, 85000, 95000
]

# Flux in cm^-2 s^-1 bin^-1 (scientific notation) from Huebner (1992)
phi = [
    0.0, 3.0e1, 2.5e3, 2.8e4, 1.8e5, 4.0e6, 4.7e7, 8.3e7, 1.03e8, 9.4e7, 1.2e8, 9.9e7, 5.6e7, 2.5e7, 1.2e7, 3.07e8, 9.0e8, 3.7e9, 1.4e9, 2.65e9, 4.5e8, 1.54e9, 5.84e9, 1.16e9, 4.5e8, 1.31e9, 1.77e8, 2.76e8, 1.77e8, 2.12e8, 4.32e8, 6.32e8, 
    2.42e8, 3.62e8, 5.17e7, 1.02e9, 5.52e8, 1.2e9, 1.15e8, 3.39e8, 1.59e8, 5.19e8, 1.28e9, 8.07e8, 1.05e9, 3.73e9, 3.34e9, 9.39e7, 2.34e8, 5.84e8, 5.94e8, 1.04e8, 1.04e8, 5.3e9, 7.56e8, 1.25e8, 1.04e8, 3.6e9, 2.46e9, 1.83e9, 1.57e8, 1.7e8, 1.31e8, 7.21e8, 
    1.31e8, 1.31e8, 1.31e8, 6.21e8, 1.31e8, 1.31e8, 9.15e7, 7.84e7, 1.03e8, 2.66e8, 1.12e8, 1.24e8, 1.82e8, 1.9e8, 7.4e8, 3.02e11, 3.67e9, 1.36e9, 1.61e9, 1.32e9, 1.41e9, 3.11e9, 1.06e9, 1.37e9, 1.02e9, 1.14e9, 7.29e9, 2.2e9, 1.59e9, 2.21e9, 1.24e9, 1.99e9, 
    3.09e9, 2.57e9, 2.74e9, 3.1e9, 7.6e9, 1.01e10, 1.3e10, 1.82e10, 1.33e10, 1.66e10, 2.9e10, 3.6e10, 4.75e10, 6.4e10, 5.49e10, 1.19e11, 1.76e11, 2.32e11, 1.44e11, 1.83e11, 1.34e11, 2.62e11, 2.88e11, 3.14e11, 3.81e11, 4.43e11, 4.95e11, 5.94e11, 6.59e11, 7.26e11, 9.85e11, 1.27e12, 
    1.01e12, 1.2e12, 1.44e12, 1.8e12, 2.08e12, 2.45e12, 5.09e12, 7.12e12, 9.23e12, 8.42e12, 1.2e13, 1.22e13, 1.77e13, 1.6e13, 1.96e13, 2.4e13, 2.25e13, 2.21e13, 2.32e13, 2.5e13, 2.73e13, 2.88e13, 3.02e13, 3.97e13, 7.13e13, 4.37e13, 1.12e14, 1.25e14, 1.16e14, 1.19e14, 1.38e14, 1.70e14,
    2.46e14, 3.90e14, 3.99e14, 3.86e14, 5.08e14, 5.92e14, 6.05e14, 6.94e14, 8.12e14, 9.71e14, 8.97e14, 9.44e14, 1.01e15, 1.03e15, 1.03e15, 1.04e15, 1.18e15, 1.23e15, 1.24e15, 1.17e15, 1.11e15, 1.09e15, 1.19e15, 1.54e15, 1.90e15, 1.99e15, 1.99e15, 2.02e15, 2.01e15, 1.94e15, 1.98e15, 2.25e15, 2.39e15, 
    2.48e15, 2.49e15, 2.48e15, 2.50e15, 2.55e15, 2.61e15, 2.59e15, 2.46e15, 2.44e15, 2.53e15, 2.48e15, 2.49e15, 2.50e15, 2.43e15, 2.43e15, 2.52e15, 2.58e15, 2.64e15, 2.67e15, 2.70e15, 2.68e15, 2.66e15, 2.66e15, 2.67e15, 2.67e15, 2.69e15, 2.71e15, 2.71e15, 2.71e15, 2.72e15, 2.72e15, 2.71e15, 2.70e15,
    2.70e15, 2.70e15, 2.69e15, 2.68e15, 2.67e15, 2.66e15, 2.65e15, 3.95e15, 5.22e15, 5.18e15, 5.14e15, 5.09e15, 5.04e15, 4.99e15, 4.94e15, 4.90e15, 4.83e15, 4.80e15, 4.67e15, 4.59e15, 4.68e15, 4.54e15, 4.60e15, 4.53e15, 4.41e15, 4.34e15, 4.31e15, 4.03e15, 4.33e15, 4.25e15, 4.27e15, 4.23e15, 4.13e15, 
    4.01e15, 3.89e15, 3.88e15, 3.78e15, 3.76e15, 3.72e15, 3.72e15, 3.79e15, 3.79e15, 7.49e15, 2.11e16, 1.68e16, 1.60e16, 1.50e16, 1.48e16, 1.43e16, 1.32e16, 1.25e16, 1.16e16, 1.12e16, 1.07e16, 9.95e15, 9.72e15, 9.20e15, 8.24e15, 7.70e15, 6.94e15, 6.51e15, 6.19e15, 7.46e15, 1.11e16, 8.28e15, 7.33e15, 
    6.83e15, 6.07e15, 5.50e15, 4.96e15, 4.51e15, 4.09e15, 3.74e15, 3.40e15, 3.18e15, 2.87e15, 2.70e15, 2.47e15, 2.31e15, 2.14e15, 2.01e15, 1.87e15, 1.71e15, 1.60e15, 1.48e15, 1.41e15, 1.33e15, 1.25e15, 1.16e15, 1.06e15, 9.67e14, 9.25e14, 2.84e15, 6.74e15, 3.35e15, 2.22e15, 1.59e15, 4.53e15
]

324-element Vector{Float64}:
      0.0
     30.0
   2500.0
  28000.0
 180000.0
      4.0e6
      4.7e7
      8.3e7
      1.03e8
      9.4e7
      ⋮
      1.06e15
      9.67e14
      9.25e14
      2.84e15
      6.74e15
      3.35e15
      2.22e15
      1.59e15
      4.53e15

#### Solar flux ratio between active and quiet sun conditions from Huebner (1992)

Extracted from Figure 3 with PlotDigitizer: https://plotdigitizer.com/

**NOTE:**
- To calculate the fluxes for active Sun conditions, we have the ration between active and quiet fluxes provided by Huebner (1992) as a plot
- As the numerical database was not found, the values from the plot were exctracted with a plot digitizer tool

In [85]:
# Wavelength vector
lambda_active = [0.00, 28.72, 53.29, 59.16, 59.16, 71.54, 71.54, 74.44, 88.08, 89.99, 
115.77, 115.77, 125.25, 125.25, 144.88, 144.88, 148.61, 148.61, 165.67, 176.91, 
223.91, 231.74, 272.68, 283.07, 298.20, 298.20, 300.99, 342.41, 344.49, 365.70, 
365.70, 366.69, 424.97, 426.87, 479.82, 480.91, 501.09, 534.21, 541.24, 580.65, 
585.59, 606.49, 609.04, 630.15, 633.66, 678.40, 683.51, 712.99, 714.07, 739.78, 
741.06, 769.47, 785.37, 796.68, 836.69, 922.60, 922.60, 961.07, 963.04, 970.42, 
1022.55, 1022.55, 1028.79, 1073.51, 1074.25, 1126.39, 1131.24, 1175.13, 1180.71, 
1184.81, 1224.35, 1271.98, 1278.65, 1300.04, 1318.69, 1326.50, 1357.78, 1364.96, 
1368.52, 1384.26, 1384.26, 1396.93, 1480.30, 1489.65, 1529.55, 1557.35, 1581.17, 
1610.21, 1615.05, 1684.84, 1687.57, 1786.33, 1918.57, 1938.65, 2034.93, 2049.99, 
2080.65, 2116.71, 2138.45, 2462.53, 2500.0, 95000.0]


# Ratio of flux active/quiet sun from Huebner (1992). Taken from Graph
ratio = [6.37, 6.37, 6.37, 2.50, 2.50, 2.32, 2.32, 2.48, 2.12, 2.31, 
2.35, 2.35, 2.58, 2.58, 3.60, 3.60, 4.51, 4.51, 4.51, 5.38, 
2.11, 5.37, 2.54, 2.10, 2.54, 2.54, 2.95, 2.14, 2.95, 2.83, 
2.83, 2.14, 2.41, 2.40, 2.40, 2.30, 2.27, 2.21, 2.01, 1.88, 
2.00, 1.79, 1.88, 1.94, 1.79, 1.94, 2.05, 2.18, 2.04, 1.93, 
2.18, 2.48, 1.91, 2.49, 2.41, 2.41, 1.97, 1.96, 2.14, 2.35, 2.36, 
1.98, 1.72, 1.72, 1.84, 1.84, 1.92, 1.84, 1.93, 2.34, 2.15, 
2.11, 1.87, 1.86, 1.88, 2.07, 1.92, 2.06, 1.99, 1.89, 1.89, 
1.65, 1.69, 1.55, 1.49, 1.48, 1.43, 1.42, 1.37, 1.26, 1.21, 
1.16, 1.15, 1.13, 1.08, 1.09, 1.05, 1.05, 1.03, 1.03, 1.01, 1.0]

102-element Vector{Float64}:
 6.37
 6.37
 6.37
 2.5
 2.5
 2.32
 2.32
 2.48
 2.12
 2.31
 ⋮
 1.13
 1.08
 1.09
 1.05
 1.05
 1.03
 1.03
 1.01
 1.0

#### Interpolated ratios for the wavelength bins given in Huebner (1992)

In [86]:
using Interpolations

In [87]:
Interpolations.deduplicate_knots!(lambda_active)

itp = interpolate((lambda_active,), ratio, Gridded(Linear()))
ratio_interp = itp(lambda_phi)

324-element Vector{Float64}:
 6.369999999999999
 6.37
 6.37
 6.370000000000001
 6.37
 6.369999999999999
 6.37
 6.37
 2.4877867528271405
 2.3423909531502423
 ⋮
 1.0052432432432432
 1.0051351351351352
 1.005027027027027
 1.004918918918919
 1.0045675675675676
 1.0032432432432432
 1.0021621621621621
 1.001081081081081
 1.0

### Method 1: Cross sectional database is available per wavelength value

Example: Helium cross-sectional data from  Marr, West (1976)

In [88]:
He_CS_data = [(505.0, 1e-20),
    (504.0, 7.56), (500.0, 7.46), (495.0, 7.33), (490.0, 7.19), (485.0, 7.06), (480.0, 6.94),
    (475.0, 6.81), (470.0, 6.68), (465.0, 6.55), (460.0, 6.43), (455.0, 6.30),
    (450.0, 6.18), (445.0, 6.05), (440.0, 5.93), (435.0, 5.81), (430.0, 5.69),
    (425.0, 5.57), (420.0, 5.45), (415.0, 5.33), (410.0, 5.21), (405.0, 5.10),
    (400.0, 4.98), (395.0, 4.87), (390.0, 4.76), (385.0, 4.64), (380.0, 4.53),
    (375.0, 4.42), (370.0, 4.31), (365.0, 4.20), (360.0, 4.09), (355.0, 4.00),
    (350.0, 3.88), (345.0, 3.78), (340.0, 3.68), (335.0, 3.57), (330.0, 3.47),
    (325.0, 3.37), (320.0, 3.27), (315.0, 3.18), (310.0, 3.08), (305.0, 2.98),
    
    (300.0, 2.89), (295.0, 2.80), (290.0, 2.70), (285.0, 2.61), (280.0, 2.52),
    (275.0, 2.44), (270.0, 2.35), (265.0, 2.26), (260.0, 2.18), (255.0, 2.10),
    (250.0, 2.02), (245.0, 1.94), (240.0, 1.86), (235.0, 1.78), (230.0, 1.70),
    (225.0, 1.63), (220.0, 1.55), (215.0, 1.48), (210.0, 1.41), (205.0, 1.34),
    (200.0, 1.28), (195.0, 1.21), (190.0, 1.14), (185.0, 1.08), (180.0, 1.02),
    (175.0, 0.961), (170.0, 0.903), (165.0, 0.847), (160.0, 0.792), (155.0, 0.738),
    (150.0, 0.687), (145.0, 0.637), (140.0, 0.588), (135.0, 0.542), (130.0, 0.497),
    (125.0, 0.454), (120.0, 0.412), (115.0, 0.373), (110.0, 0.335), (105.0, 0.299), (100.0, 0.265), 
    
    (95.0, 0.233), (90.0, 0.202), ( 85.0, 0.174), ( 80.0, 0.147), ( 75.0, 0.123), 
    (70.0, 0.100), (65.0, 0.0795), ( 60.0, 0.0609),( 55.0, 0.0443), ( 50.0, 0.0315), 
    (48.0, 0.0282), (46.0, 0.0250),  (44.0, 0.0220), (42.0, 0.0193), (40.0, 0.0168), 
    (38.0, 0.0145), (36.0, 0.0124), (34.0, 0.0105), (32.0, 0.00885), (30.0, 0.00736), 
    (28.0, 0.00604), (26.0, 0.00489), (24.0, 0.00389), (22.0, 0.00303), (20.0, 0.00231), 
    (19.0, 0.00199), (18.0, 0.00171), (17.0, 0.00145), (16.0, 0.00122),(15.0, 0.00101), 
    (14.0, 0.000832), (13.0, 0.000673), (12.0, 0.000535), (11.0, 0.000417), (10.0, 0.000318), 
    (9.5, 0.000274), (9.0, 0.000235), (8.5, 0.000200), (8.0, 0.000168), (7.5, 0.000139), (7.0, 0.000115), 
    
    (6.5, 0.000093), (6.0, 0.000074), (5.5, 0.000057), (5.0, 0.000044), (4.5, 0.000031), 
    (4.0, 0.000023), (3.5, 0.000016), (3.0, 0.000010), (2.5, 0.000006), (2.0, 0.000003), (1.5, 0.000001), (0.0, 1e-20)
]

# Separate into two Python lists: wavelengths and cross sections
wvl = reverse([r[1] for r in He_CS_data])
cs  = reverse([r[2] for r in He_CS_data])

136-element Vector{Float64}:
 1.0e-20
 1.0e-6
 3.0e-6
 6.0e-6
 1.0e-5
 1.6e-5
 2.3e-5
 3.1e-5
 4.4e-5
 5.7e-5
 ⋮
 6.68
 6.81
 6.94
 7.06
 7.19
 7.33
 7.46
 7.56
 1.0e-20

#### Interpolate cross sectional data to have it in the same reference wvl as the solar flux

In [89]:
itp = LinearInterpolation(wvl, cs, extrapolation_bc=Flat())  
cs_interp = itp.(lambda_phi)

324-element Vector{Float64}:
 6.666666666666699e-7
 3.0e-6
 2.3e-5
 7.4e-5
 0.000168
 0.000318
 0.0168
 0.0315
 0.0609
 0.1
 ⋮
 1.0e-20
 1.0e-20
 1.0e-20
 1.0e-20
 1.0e-20
 1.0e-20
 1.0e-20
 1.0e-20
 1.0e-20

#### Quiet Sun

Calculate flux in photons/second and print photodestruction rate

In [90]:
He_flux_quiet = cs_interp .* 1e-18 .* phi

println("Photodestruction rate: ", sum(He_flux_quiet))

Photodestruction rate: 5.5744213704376146e-8


Print flux and copy it to `solar_database.jl` accordingly

In [91]:
print(Float32.(Tuple(He_flux_quiet)))

(0.0f0, 9.0f-23, 5.75f-20, 2.072f-18, 3.024f-17, 1.272f-15, 7.896f-13, 2.6145f-12, 6.2727f-12, 9.4f-12, 1.764f-11, 1.9998f-11, 1.484f-11, 8.375f-12, 4.944f-12, 1.472986f-10, 6.4584f-10, 3.59936f-9, 1.876f-9, 4.5474f-9, 1.0575f-9, 3.8808f-9, 1.68776f-8, 3.7932f-9, 1.656f-9, 5.3579f-9, 7.6287f-10, 1.37448f-9, 1.00713f-9, 1.36316f-9, 2.99808f-9, 4.71472f-9, 2.42f-30, 3.62f-30, 5.17f-31, 1.02f-29, 5.52f-30, 1.2f-29, 1.15f-30, 3.39f-30, 1.59f-30, 5.19f-30, 1.28f-29, 8.07f-30, 1.05f-29, 3.73f-29, 3.34f-29, 9.39f-31, 2.34f-30, 5.84f-30, 5.94f-30, 1.04f-30, 1.04f-30, 5.3f-29, 7.56f-30, 1.25f-30, 1.04f-30, 3.6f-29, 2.46f-29, 1.83f-29, 1.57f-30, 1.7f-30, 1.31f-30, 7.21f-30, 1.31f-30, 1.31f-30, 1.31f-30, 6.21f-30, 1.31f-30, 1.31f-30, 9.15f-31, 7.84f-31, 1.03f-30, 2.66f-30, 1.12f-30, 1.24f-30, 1.82f-30, 1.9f-30, 7.4f-30, 3.02f-27, 3.67f-29, 1.36f-29, 1.61f-29, 1.32f-29, 1.41f-29, 3.11f-29, 1.06f-29, 1.37f-29, 1.02f-29, 1.14f-29, 7.29f-29, 2.2f-29, 1.59f-29, 2.21f-29, 1.24f-29, 1.99f-29, 3.09f-29, 

Calculate the normalized flux values when considering the full electromagnetic spectrum. This is the weight of a certain photon having a certain wavelength/energy per unit time. 
Print normalized fluxes and modify `solar_database.jl` accordingly

In [92]:
normalize_flux_distribution(lambda_phi, He_flux_quiet, (0.0, 95000))[2]

(0.0f0, 1.6145173f-15, 1.0314972f-12, 3.7169778f-11, 5.4247784f-10, 2.2818512f-8, 1.4164699f-5, 4.690173f-5, 0.000112526475f0, 0.00016862736f0, 0.00031644542f0, 0.00035874575f0, 0.000266216f0, 0.00015023982f0, 8.8690824f-5, 0.0026424017f0, 0.011585777f0, 0.06456921f0, 0.033653717f0, 0.081576176f0, 0.018970579f0, 0.069617994f0, 0.30276865f0, 0.068046525f0, 0.029707119f0, 0.096115805f0, 0.013685187f0, 0.024656909f0, 0.01806699f0, 0.02445384f0, 0.053782802f0, 0.084577754f0, 4.3412578f-23, 6.493948f-23, 9.274505f-24, 1.8297865f-22, 9.9023734f-23, 2.1526898f-22, 2.0629945f-23, 6.081349f-23, 2.852314f-23, 9.310384f-23, 2.2962024f-22, 1.447684f-22, 1.8836036f-22, 6.6912777f-22, 5.9916533f-22, 1.6844798f-23, 4.197745f-23, 1.0476424f-22, 1.0655814f-22, 1.8656646f-23, 1.8656646f-23, 9.507713f-22, 1.3561946f-22, 2.2423852f-23, 1.8656646f-23, 6.45807f-22, 4.413014f-22, 3.2828519f-22, 2.816436f-23, 3.0496438f-23, 2.3500196f-23, 1.2934079f-22, 2.3500196f-23, 2.3500196f-23, 2.3500196f-23, 1.114017f-2

#### Active Sun

Calculate flux in photons/second and print photodestruction rate

In [93]:
He_flux_active = cs_interp .* 1e-18 .* phi .* ratio_interp

print("Photodestruction rate:", sum(He_flux_active))


Photodestruction rate:1.6803216302077845e-7

Print flux and copy it to `solar_database.jl` accordingly

In [94]:
print(Float32.(Tuple(He_flux_active)))

(0.0f0, 5.733f-22, 3.66275f-19, 1.319864f-17, 1.926288f-16, 8.10264f-15, 5.029752f-12, 1.6654365f-11, 1.560514f-11, 2.2018476f-11, 4.115862f-11, 4.619569f-11, 3.4510887f-11, 1.9606271f-11, 1.2125786f-11, 4.010784f-10, 2.9127385f-9, 1.9111033f-8, 6.4265255f-9, 2.3018497f-8, 2.8819587f-9, 8.654221f-9, 4.73335f-8, 9.779798f-9, 3.6218863f-9, 1.5335644f-8, 1.6442401f-9, 3.1534952f-9, 2.417112f-9, 3.271584f-9, 7.1458826f-9, 1.0710054f-8, 5.4104973f-30, 7.4039044f-30, 1.0071766f-30, 1.919787f-29, 1.0240762f-29, 2.2933813f-29, 2.2305097f-30, 6.69336f-30, 3.4452355f-30, 1.02397076f-29, 3.15008f-29, 2.0040729f-29, 2.5445456f-29, 8.9893f-29, 8.0494f-29, 2.26299f-30, 5.6394f-30, 1.1493566f-29, 1.1674933f-29, 2.0413926f-30, 2.0386892f-30, 1.2391658f-28, 1.7779893f-29, 2.9433938f-30, 2.4512977f-30, 8.492167f-29, 4.414675f-29, 3.1476f-29, 2.7004f-30, 2.924f-30, 2.2532f-30, 1.32664f-29, 2.4104f-30, 2.4104f-30, 2.4104f-30, 1.14264f-29, 2.4884057f-30, 2.494283f-30, 1.725512f-30, 1.4684683f-30, 1.917973f

Calculate the normalized flux values when considering the full electromagnetic spectrum. This is the weight of a certain photon having a certain wavelength/energy per unit time. 
Print normalized fluxes and modify `solar_spectrum.jl` accordingly

In [95]:
normalize_flux_distribution(lambda_phi, He_flux_active, (0.0, 95000))[2]

(0.0f0, 3.4118468f-15, 2.179791f-12, 7.85483f-11, 1.1463805f-9, 4.822077f-8, 2.993327f-5, 9.911416f-5, 9.286997f-5, 0.00013103728f0, 0.0002449449f0, 0.0002749217f0, 0.00020538263f0, 0.00011668166f0, 7.216348f-5, 0.0023869146f0, 0.01733441f0, 0.11373438f0, 0.03824581f0, 0.13698864f0, 0.017151233f0, 0.051503364f0, 0.2816931f0, 0.058201943f0, 0.021554722f0, 0.091266125f0, 0.00978527f0, 0.018767213f0, 0.014384818f0, 0.019469986f0, 0.042526875f0, 0.06373812f0, 3.219918f-23, 4.4062425f-23, 5.9939517f-24, 1.1425117f-22, 6.0945254f-23, 1.3648466f-22, 1.3274303f-23, 3.9833803f-23, 2.0503429f-23, 6.0938975f-23, 1.8746887f-22, 1.1926722f-22, 1.5143206f-22, 5.3497497f-22, 4.7903926f-22, 1.34676f-23, 3.3561434f-23, 6.840099f-23, 6.948035f-23, 1.2148821f-23, 1.2132732f-23, 7.3745757f-22, 1.05812444f-22, 1.7516849f-23, 1.4588266f-23, 5.053894f-22, 2.62728f-22, 1.8732128f-22, 1.6070733f-23, 1.740143f-23, 1.3409339f-23, 7.8951553f-23, 1.4344874f-23, 1.4344874f-23, 1.4344874f-23, 6.800127f-23, 1.4809104

### Method 2: Direct Photon Flux Data per Second and per Wavelength Bin 
*(Multiplication by cross section has already been performed)*

In the past, a cross-sectional database for all the species analyzed in Huebner (2015) was available at [http://phidrates.space.swri.edu/](http://phidrates.space.swri.edu/).  

This website appears to no longer be publicly accessible. Cross-sectional data may still be found in other publications (see Method 1).  

If these sources are not available, Huebner (1992) provides plots of the flux of solar photons per second and per wavelength bin for individual species and reactions. This data already accounts for the product of the species- and wavelength-dependent cross section with the solar flux, and it can be extracted using tools such as [Plot Digitizer](https://plotdigitizer.com/app).  

**Example:** Helium flux in photons/s per wavelength bin from Huebner (1992)

#### Quiet Sun

- Flux data should be present in csv file with 2 columns: 'Wavelength (A)' and 'Flux(photon.s-1.binwvl)'
- Make sure data is correctly sorted (increasing wvl)

In [96]:
using CSV, DataFrames

In [97]:
df = CSV.read("Helium_quiet.csv", DataFrame)

Row,Wavelength (A),Flux(photon.s-1.binwvl)
,Float64,Float64
1,7.16612,9.7972e-18
2,7.47511,1.14935e-17
3,10.2751,1.32221e-17
4,10.6092,5.68715e-16
5,40.4148,5.65023e-16
6,40.4718,8.21086e-14
7,50.0411,8.21086e-14
8,50.6779,2.53359e-13
9,60.9615,2.53359e-13


Interpolate flux in photons/s to match with Huebner 1992 standard wavelength values

Check for duplicates or non sorted values

In [98]:
wvl = df[!, "Wavelength (A)"]

# Find duplicates
duplicates = wvl[findall(x -> count(==(x), wvl) > 1, wvl)]
println("Duplicates: ", unique(duplicates))

# Find elements that are not strictly increasing
unsorted_indices = findall(diff(wvl) .<= 0)
println("Indices of elements not strictly increasing: ", unsorted_indices)
println("Values at these indices: ", wvl[unsorted_indices], " and next values: ", wvl[unsorted_indices .+ 1])

Duplicates: Float64[]
Indices of elements not strictly increasing: Int64[]
Values at these indices: Float64[] and next values: Float64[]


In [99]:
itp = LinearInterpolation(df[!, "Wavelength (A)"], df[!, "Flux(photon.s-1.binwvl)"], extrapolation_bc=Flat())
He_flux = itp.(lambda_phi)

324-element Vector{Float64}:
 9.797200108554883e-18
 9.797200108554883e-18
 9.797200108554883e-18
 9.797200108554883e-18
 1.1817507077368749e-17
 1.3052214521847203e-17
 5.65074596483053e-16
 8.210861933545769e-14
 2.533593257815089e-13
 5.08905473854394e-13
 ⋮
 9.64651639764944e-18
 9.64651639764944e-18
 9.64651639764944e-18
 9.64651639764944e-18
 9.64651639764944e-18
 9.64651639764944e-18
 9.64651639764944e-18
 9.64651639764944e-18
 9.64651639764944e-18

Calculate photodestruction: sum flux contributions multiplied by wavelength bin

In [100]:
λ_shift .- lambda_phi

324-element Vector{Float64}:
     1.0
     2.0
     2.0
     2.0
     2.0
    30.0
    10.0
    10.0
    10.0
    10.0
     ⋮
  1000.0
  1000.0
  1000.0
  3250.0
 12250.0
 10000.0
 10000.0
 10000.0
   NaN

In [101]:
λ_shift = vcat(lambda_phi[2:end], NaN)
flux_times_bin = He_flux .* (λ_shift .- lambda_phi)

# Sum while ignoring NaN (exactly like pandas skipna=True)
photodestruction_rate = sum(filter(!isnan, flux_times_bin))

println("Photodestruction rate: ", photodestruction_rate)

Photodestruction rate: 4.195679304392522e-8


Print flux and copy it to `solar_database.jl` accordingly

In [102]:
Float32.(Tuple(He_flux))

(9.7972f-18, 9.7972f-18, 9.7972f-18, 9.7972f-18, 1.1817507f-17, 1.3052214f-17, 5.650746f-16, 8.2108616f-14, 2.5335932f-13, 5.0890547f-13, 9.022659f-13, 1.2219256f-12, 1.2173062f-12, 9.571613f-13, 5.1426593f-13, 3.9615248f-13, 4.5515475f-12, 6.750455f-11, 8.836555f-11, 5.9420434f-11, 9.9880604f-11, 1.5514351f-10, 4.3476417f-10, 1.9365753f-10, 9.087649f-11, 5.0224297f-10, 8.134734f-11, 5.3246917f-11, 4.0324966f-11, 5.691496f-11, 1.9552118f-10, 7.0341975f-11, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.646517f-18, 9.64651

Calculate the normalized flux values when considering the full electromagnetic spectrum. This is the weight of a certain photon having a certain wavelength/energy per unit time. 
Print normalized fluxes and modify `solar_database.jl` accordingly

In [103]:
normalize_flux_distribution(lambda_phi, He_flux, (0.0, 95000))[2]

(4.452946f-9, 4.452946f-9, 4.452946f-9, 4.452946f-9, 5.3712f-9, 5.9323897f-9, 2.5683323f-7, 3.731936f-5, 0.00011515488f0, 0.0002313037f0, 0.00041009078f0, 0.00055537996f0, 0.0005532804f0, 0.0004350414f0, 0.0002337401f0, 0.0001800561f0, 0.0020687336f0, 0.030681636f0, 0.04016321f0, 0.027007308f0, 0.045396943f0, 0.070514604f0, 0.19760558f0, 0.08801969f0, 0.041304465f0, 0.22827551f0, 0.03697335f0, 0.024201369f0, 0.018328186f0, 0.025868539f0, 0.08886675f0, 0.03197128f0, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9, 4.3844586f-9

#### Repeat for Active Sun